# Population <-> Cities

In [11]:
# To hide the warnings
import warnings
warnings.filterwarnings('ignore')

import pandas as pd 
import networkx as nx
import numpy as np
import pickle
import matplotlib.pyplot as plt

In [8]:
data_stop_detail = pd.read_csv('stops_details.csv', sep=',', encoding='latin-1')
del data_stop_detail['Unnamed: 0']

data_stop_detail.columns = ['trip_id', 'stop_id', 'stop_name', 'seq']

data_stop_detail

,trip_id,stop_id,stop_name,seq
0,1.TA.1-1-A-j18-1.1.R,8500010:0:3,Basel SBB,1.0
1,1.TA.1-1-A-j18-1.1.R,8500020:0:3,Muttenz,2.0
2,1.TA.1-1-A-j18-1.1.R,8500021:0:5,Pratteln,3.0
3,1.TA.1-1-A-j18-1.1.R,8517131:0:2,Pratteln Salina Raurica,4.0
4,1.TA.1-1-A-j18-1.1.R,8500300:0:5,Kaiseraugst,5.0
5,1.TA.1-1-A-j18-1.1.R,8500313:0:2,Rheinfelden Augarten,6.0
6,1.TA.1-1-A-j18-1.1.R,8500301:0:3,Rheinfelden,7.0
7,1.TA.1-1-A-j18-1.1.R,8500302:0:3,Mohlin,8.0
8,1.TA.1-1-A-j18-1.1.R,8500303:0:2,Mumpf,9.0
9,1.TA.1-1-A-j18-1.1.R,8500320:0:3,Stein-Säckingen,10.0


In [17]:
with open('swiss_population_pickle.pkl', 'rb') as f:
    population = pickle.load(f)

In [24]:
test_data = data_stop_detail.copy()

success = 0
failure = 0

for idx, row in test_data.iterrows():
    try:
        pop = population[row['stop_name']]
        success += 1
    except:
        failure += 1
        
print("Found: {} ==== Not found: {}".format(success, failure))

Found: 90482 ==== Not found: 165458


In [42]:
def alt_city_names(city_orig_name):
    stop_extensions = ['HB', 'Hbf', 'SBB', 'CFF']
    return ["{} {}".format(city_orig_name, ext) for ext in stop_extensions]

In [67]:
POP_LIMIT_LARGE_CITY = 24000
large_cities = [city for city, pop in population.items() if pop > POP_LIMIT_LARGE_CITY]
print(len(large_cities))




31


In [66]:
def real_stop_names(cities, stop_names):
    city_stop_names = []
    
    for city in cities:
        found = False
        if city in stop_names.values:
            found = True
            city_stop_names.append(city)

        if not found:
            for alt_name in alt_city_names(city):
                if alt_name in stop_names.values:
                    found = True
                    city_stop_names.append(alt_name)
                    break
                    
        if not found:
            if city == "Fribourg":
                city_stop_names.append("Fribourg/Freiburg")
            if city == "Rapperswil-Jona":
                city_stop_names.append("Rapperswil")

    return(city_stop_names)

In [80]:
df = data_stop_detail

result = real_stop_names(large_cities, pd.Series(df['stop_name']))

line = df.loc[df['stop_name'] == result[0]]
stop_lat = 


,trip_id,stop_id,stop_name,seq
111821,1.TA.26-14-j18-1.1.R,8503128:0:4,Dübendorf,4.0
111831,3.TA.26-14-j18-1.1.R,8503128:0:4,Dübendorf,4.0
111841,2.TA.26-14-j18-1.2.R,8503128:0:4,Dübendorf,4.0
111851,4.TA.26-14-j18-1.2.R,8503128:0:4,Dübendorf,4.0
111861,5.TA.26-14-j18-1.2.R,8503128:0:4,Dübendorf,4.0
111871,6.TA.26-14-j18-1.2.R,8503128:0:4,Dübendorf,4.0
111881,7.TA.26-14-j18-1.2.R,8503128:0:4,Dübendorf,4.0
111891,8.TA.26-14-j18-1.2.R,8503128:0:4,Dübendorf,4.0
111901,9.TA.26-14-j18-1.2.R,8503128:0:4,Dübendorf,4.0
111917,10.TA.26-14-j18-1.3.H,8503128:0:3,Dübendorf,10.0


In [74]:
len(population)

2287